In [ ]:
import pandas as pd
import pyodbc
import math
import os
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential
import struct

# تحميل المتغيرات من ملف .env
load_dotenv()

# إعداد الاتصال
tenant_id = os.getenv("TENANT_ID")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
server = os.getenv("DB_SERVER")
database = os.getenv("DB_DATABASE")
driver = "{ODBC Driver 17 for SQL Server}"

# الحصول على Access Token من Azure AD
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://database.windows.net/.default")

# تحويل الـ token إلى شكل مناسب لـ pyodbc
exptoken = bytes(token.token, "UTF-8")
tokenstruct = struct.pack(f"<I{len(exptoken)}s", len(exptoken), exptoken)

# تحميل بيانات CSV
df = pd.read_csv("cleaned_flights.csv")

# تجهيز الأعمدة المطلوبة فقط
df = df[[
    'flight_number', 'airport_code', 'departure_arrival_time_local',
    'terminal', 'aircraft_model', 'airline_name', 'type',
    'scheduled_datetime_flight', 'flight_status', 'gate_status', 'gate_number'
]]

# الاتصال بقاعدة البيانات باستخدام Azure AD Token
conn = pyodbc.connect(
    f"DRIVER={driver};SERVER={server};DATABASE={database};Encrypt=yes;TrustServerCertificate=no;Authentication=ActiveDirectoryAccessToken;",
    attrs_before={1256: tokenstruct}
)

cursor = conn.cursor()
cursor.fast_executemany = True

# أمر الإدخال
insert_sql = """
    INSERT INTO Flights (
        flight_number, airport_code, departure_arrival_time_local,
        terminal, aircraft_model, airline_name, type,
        scheduled_datetime_flight, flight_status, gate_status, gate_number
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# تحويل البيانات إلى قائمة
records = df.to_records(index=False)
records_list = list(records)

# الإدخال على دفعات
batch_size = 500
total_batches = math.ceil(len(records_list) / batch_size)

for i in range(total_batches):
    start = i * batch_size
    end = start + batch_size
    batch = records_list[start:end]
    cursor.executemany(insert_sql, batch)
    conn.commit()
    print(f"✅ Batch {i+1}/{total_batches} inserted")

cursor.close()
conn.close()
print("✅ All data imported successfully using Azure AD authentication.")
